In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys
parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)
%matplotlib widget

In [ ]:
from LinearMPC_4_1.MPCVelControl import MPCVelControl
from PIControl.PIControl import PIControl
import numpy as np
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis, plot_static_states_inputs

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:
Ts = 0.05
sim_time = 20
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])
pos_target = np.array([0, 0, 10.0])

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller = PIControl(pos_target)
mpc = MPCVelControl.new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, pos_control=pos_controller, method="nonlinear"
)

# Print final results
print(f"\nFinal Results:")
print(f"  Position:")
print(f"    x = {x_cl[9, -1]:.3f} m (target: {pos_target[0]:.1f} m)")
print(f"    y = {x_cl[10, -1]:.3f} m (target: {pos_target[1]:.1f} m)")
print(f"    z = {x_cl[11, -1]:.3f} m (target: {pos_target[2]:.1f} m)")
print(f"  Velocity:")
print(f"    vx = {x_cl[6, -1]:.3f} m/s")
print(f"    vy = {x_cl[7, -1]:.3f} m/s")
print(f"    vz = {x_cl[8, -1]:.3f} m/s")
print(f"  Orientation:")
print(f"    alpha = {np.rad2deg(x_cl[3, -1]):.2f}°")
print(f"    beta = {np.rad2deg(x_cl[4, -1]):.2f}°")
print(f"    gamma = {np.rad2deg(x_cl[5, -1]):.2f}°")

vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 1.0
vis.animate(t_cl[:-1], x_cl[:, :-1], u_cl, Ref=ref[:, :-1], T_ol=t_ol[..., :-1], X_ol=x_ol, U_ol=u_ol)

# Display static plots
plot_static_states_inputs(t_cl[:-1], x_cl[:, :-1], u_cl, ref[:, :-1])